In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
from sklearn.preprocessing import MinMaxScaler

In [2]:
covidNumbers = pd.read_csv('Data/owid-covid-data.csv')
covidNumbers.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Data/owid-covid-data.csv'

In [ ]:
print(covidNumbers['date'].dtypes)
covidNumbers['date'] = pd.to_datetime(covidNumbers['date'], format = '%Y-%m-%d')

In [ ]:
covidUsa = covidNumbers[covidNumbers['iso_code'] == 'USA']
covidUsa.head()

In [ ]:
stringencyData = pd.read_csv('Data/covid-stringency-index.csv')
stringencyData.head()

In [ ]:
stringencyData['Date'] = pd.to_datetime(stringencyData['Date'], format = '%Y-%m-%d')

In [ ]:
stringencyUsa = stringencyData[stringencyData['Code'] == 'USA']
stringencyUsa.tail()

In [ ]:
stringencyUsa.rename(columns={"Date": "date"}, inplace=True)
casesStringency = pd.merge(covidUsa[['date', 'new_cases_smoothed']].copy(), 
                           stringencyUsa[['date', 'stringency_index']].copy(), on='date')
casesStringency.fillna(0, inplace=True)

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=casesStringency['date'], y=casesStringency['new_cases_smoothed'], name="# New Cases"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=casesStringency['date'], y=casesStringency['stringency_index'], name="Stringency Index"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Comparison between cases and stringency",
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    yaxis=dict(
        title="# New Cases",
        titlefont=dict(
            color="#636efa"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="Stringency Index",
        titlefont=dict(
            color="#ef553c"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="# New Cases", secondary_y=False)
fig.update_yaxes(title_text="Stringency index", secondary_y=True)

fig.show()

In [ ]:
industry = pd.read_csv('Data/Stocks/sm_dj_industry.csv')
pharma = pd.read_csv('Data/Stocks/sm_dj_pharma.csv')
tourism = pd.read_csv('Data/Stocks/sm_dj_tourism.csv')
sp500 = pd.read_csv('Data/Stocks/sm_sp500.csv')
hero = pd.read_csv('Data/Stocks/sm_hero.csv')
nasdaq100 = pd.read_csv('Data/Stocks/sm_nasdaq.csv')
retail = pd.read_csv('Data/Stocks/sm_dj_retail.csv')
vix = pd.read_csv('Data/vix.csv', index_col = 0)

In [ ]:
industry.dtypes

In [ ]:
vix.reset_index(inplace=True)
vix['Data'] = pd.to_datetime(vix['Data'], format = '%d.%m.%Y')

In [ ]:
industry.head()

In [ ]:
# stocksConcatList= [pharma, tourism, sp500, hero, nasdaq100, retail, vix]

casesStringency = industry[['Data', 'Último']].copy()
casesStringency.rename(columns={"Último": "industry"}, inplace=True)
casesStringency['Data'] = pd.to_datetime(casesStringency['Data'], format = '%d.%m.%Y')

pharma.rename(columns={"Último": "pharma"}, inplace=True)
pharma['Data'] = pd.to_datetime(pharma['Data'], format = '%d.%m.%Y')
casesStringency = pd.merge(casesStringency, pharma[['Data', 'pharma']].copy(), on='Data')

tourism.rename(columns={"Último": "tourism"}, inplace=True)
tourism['Data'] = pd.to_datetime(tourism['Data'], format = '%d.%m.%Y')
casesStringency = pd.merge(casesStringency, tourism[['Data', 'tourism']].copy(), on='Data')

sp500.rename(columns={"Último": "sp500"}, inplace=True)
sp500['Data'] = pd.to_datetime(sp500['Data'], format = '%d.%m.%Y')
casesStringency = pd.merge(casesStringency, sp500[['Data', 'sp500']].copy(), on='Data')

hero.rename(columns={"Último": "hero"}, inplace=True)
hero['Data'] = pd.to_datetime(hero['Data'], format = '%d.%m.%Y')
casesStringency = pd.merge(casesStringency, hero[['Data', 'hero']].copy(), how='outer', on='Data')

nasdaq100.rename(columns={"Último": "nasdaq100"}, inplace=True)
nasdaq100['Data'] = pd.to_datetime(nasdaq100['Data'], format = '%d.%m.%Y')
casesStringency = pd.merge(casesStringency, nasdaq100[['Data', 'nasdaq100']].copy(), on='Data')

retail.rename(columns={"Último": "retail"}, inplace=True)
retail['Data'] = pd.to_datetime(retail['Data'], format = '%d.%m.%Y')
casesStringency = pd.merge(casesStringency, retail[['Data', 'retail']].copy(), on='Data')

vix.rename(columns={"Último": "vix"}, inplace=True)
vix['Data'] = pd.to_datetime(vix['Data'], format = '%d.%m.%Y')
casesStringency = pd.merge(casesStringency, vix[['Data', 'vix']].copy(), on='Data')

In [ ]:
casesStringency.set_index('Data', inplace=True)
casesStringency.tail()

In [ ]:
sp500 = sp500[['Data', 'sp500']]
sp500 = sp500[(sp500['Data'] > '2020-01-22') & (sp500['Data'] < '2021-02-22')]
sp500['sp500'] = (sp500['sp500'].replace('\.','', regex=True)
                        .replace(',','.', regex=True)
                        .astype(float))
sp500.head()

In [ ]:
data_sectors = [dict(type='scatter',
                     x=sp500.Data,
                     y=sp500['sp500'])]

layout_sectors = dict(title=dict(text='SP500 stock price'),
                      xaxis=dict(title='Date'),
                      yaxis=dict(title='Price'),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)',)

fig_sectors = go.Figure(data=data_sectors, layout=layout_sectors)

fig_sectors.show()

In [ ]:
sectors_list = casesStringency.columns
scaler = MinMaxScaler()
for s in sectors_list:
    casesStringency[s] = (casesStringency[s].replace('\.','', regex=True)
                        .replace(',','.', regex=True)
                        .astype(float))
    casesStringency[s] = scaler.fit_transform(casesStringency[s].values.reshape(-1, 1))

In [ ]:
sectors_list = casesStringency.columns

data_sectors = [dict(type='scatter',
                     x=casesStringency.index,
                     y=casesStringency[sector],
                     name=sector)
                            for sector in sectors_list]

layout_sectors = dict(title=dict(text='Sectors stock price'),
                      xaxis=dict(title='Date'),
                      yaxis=dict(title='Price'),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)',)

fig_sectors = go.Figure(data=data_sectors, layout=layout_sectors)

fig_sectors.show()

In [ ]:
casesStringency

In [ ]:
stockVsCovid = pd.read_csv('Data/StockVsCovid.csv')
geoCovid = pd.read_csv('Data/geoMap-Covid.csv')
geoStock = pd.read_csv('Data/geoMap-Stock.csv')

In [ ]:
stockVsCovid.reset_index(level=1, inplace=True)
stockVsCovid.rename(columns={"level_1": "Stock", 'Categoria: Todas as categorias': 'Covid'}, inplace=True)
stockVsCovid = stockVsCovid.iloc[3:]

In [ ]:
stockVsCovid = stockVsCovid.astype('int32').copy()

In [ ]:
stockVsCovid_list = stockVsCovid.columns

data_stockVsCovid = [dict(type='scatter',
                     x=stockVsCovid.index,
                     y=stockVsCovid[sc],
                     name=sc)
                            for sc in stockVsCovid_list]

layout_stockVsCovid = dict(title=dict(text='Trends in interests'),
                      xaxis=dict(title='Date'),
                      yaxis=dict(title='Interest'),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)',)

fig_sectors = go.Figure(data=data_stockVsCovid, layout=layout_stockVsCovid)

fig_sectors.show()

In [ ]:
geoCovid.head()

In [ ]:
geoCovid.reset_index(level=0, inplace=True)
geoCovid.rename(columns={"Category: All categories": "Interest", "index": "state"}, inplace=True)
geoCovid = geoCovid.iloc[1:]

In [ ]:
df_locations = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
df_locations.head()

In [ ]:
df_us_covid = pd.merge(geoCovid, df_locations[['code', 'state']].copy(), on='state')
df_us_covid.shape

In [ ]:
df_us_covid.head()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=df_us_covid['code'],
    z = df_us_covid['Interest'].astype(float),
    locationmode = 'USA-states',
    colorbar_title = "Interest",
    colorscale = 'Reds'
))

fig.update_layout(
    title_text = "Google trends interest on 'Covid'",
    geo_scope='usa',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()

In [ ]:
geoStock.reset_index(level=0, inplace=True)
geoStock.rename(columns={"Category: All categories": "Interest", "index": "state"}, inplace=True)
geoStock = geoStock.iloc[1:]
geoStock.head()

In [ ]:
df_us_stock = pd.merge(geoStock, df_locations[['code', 'state']].copy(), on='state')
df_us_stock.shape

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=df_us_stock['code'],
    z = df_us_stock['Interest'].astype(float),
    locationmode = 'USA-states',
    colorbar_title = "Interest",
    colorscale = 'Blues'
))

fig.update_layout(
    title_text = "Google trends interest on 'Stocks'",
    geo_scope='usa',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()

In [ ]:
trendCovid19_20 = pd.read_csv('Data/multiTimeline-stock19_20.csv')

In [ ]:
trendCovid19_20.head()

In [ ]:
trendCovid19_20.reset_index(inplace=True)
trendCovid19_20.rename(columns={"index": "date", "Category: All categories": "trendRating"}, inplace=True)
trendCovid19_20 = trendCovid19_20.iloc[1:]

trendCovid19_20['trendRating'] = trendCovid19_20['trendRating'].astype('int32').copy()
trendCovid19_20['date'] = pd.to_datetime(trendCovid19_20['date'], format = '%Y-%m-%d')

In [ ]:
mean19 = trendCovid19_20[trendCovid19_20['date'].dt.year == 2019]['trendRating'].mean()
mean20 = trendCovid19_20[trendCovid19_20['date'].dt.year == 2020]['trendRating'].mean()
print(mean19)
print(mean20)
ratio = mean19/mean20
print(ratio)
trendCovid19_20.head()

In [ ]:
geoStock19 = pd.read_csv('Data/geoMap-Stock19.csv')
geoStock20 = pd.read_csv('Data/geoMap-Stock20.csv')

In [ ]:
geoStock19.head()

In [ ]:
geoStock19.reset_index(level=0, inplace=True)
geoStock19.rename(columns={"Category: All categories": "Interest", "index": "state"}, inplace=True)
geoStock19 = geoStock19.iloc[1:]
geoStock19['Interest'] = geoStock19['Interest'].astype('int32').copy()

geoStock19 = pd.merge(geoStock19, df_locations[['code', 'state']].copy(), on='state')
geoStock19.head()

In [ ]:
geoStock19['Interest'] = geoStock19['Interest']*ratio
geoStock19.head()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=geoStock19['code'],
    z = geoStock19['Interest'].astype(float),
    locationmode = 'USA-states',
    colorbar_title = "Interest",
    colorscale = 'YlGnBu',
    zmin=0, zmax=100,
))

fig.update_layout(
    title_text = "Google trends interest on 'Stocks' in 2019",
    geo_scope='usa',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()

In [ ]:
geoStock20.reset_index(level=0, inplace=True)
geoStock20.rename(columns={"Category: All categories": "Interest", "index": "state"}, inplace=True)
geoStock20 = geoStock20.iloc[1:]
geoStock20['Interest'] = geoStock20['Interest'].astype('int32').copy()

geoStock20 = pd.merge(geoStock20, df_locations[['code', 'state']].copy(), on='state')
geoStock20.head()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=geoStock20['code'],
    z = geoStock20['Interest'].astype(float),
    locationmode = 'USA-states',
    colorbar_title = "Interest",
    colorscale = 'YlGnBu',
    zmin=0, zmax=100
))

fig.update_layout(
    title_text = "Google trends interest on 'Stocks' in 2020",
    geo_scope='usa',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()